# epaaws example

---
    author: Barron H. Henderson
    date: 2024-04-25
    last-updated: 2024-04-25
---

This shows how to use epaaws to acquire a file, investigate it, and make a plot.

In [ ]:
!python -m pip install -qq epaaws pycno  # install libraries

In [ ]:
# import modeling platform 2022
from epaaws import mp2022

In [ ]:
# open a GRIDCRO2D file
f = mp2022.open_gridcro2d('2022-01-01')
f

In [ ]:
levels = [-100, -10, 1, 100, 200, 400, 800, 1600, 3200]
qm = f['HT'].plot(levels=levels, cmap='terrain')
f.csp.cno.drawstates()

In [ ]:
print('\n - '.join(['Other files currently accessible:'] + mp2022.__all__))

In [ ]:
lf = mp2022.open_lufrac_cro('2022-07-15')
lf

# LUFRAC has types in LAY

Numeric indices are stored in VGLVLS and described in 
https://github.com/wrf-model/WRF/blob/master/run/LANDUSE.TBL#L503-L545

In [ ]:
lf.coords['LAY'] = lf.VGLVLS[:-1].astype('<d')
Z = lf['LUFRAC'].sel(LAY=[1, 2, 3, 4, 5, 28, 29, 30]).mean('LAY')
Z.attrs.update(long_name='All Forests', units='1')

Z.plot()
lf.csp.cno.drawstates(resnum=1)